In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
titanic = sns.load_dataset('titanic')

In [3]:
titanic.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [4]:
titanic.groupby(['pclass','age']).mean().fare

/tmp/ipykernel_1694/1775159437.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  titanic.groupby(['pclass','age']).mean().fare


pclass  age  
1       0.92     151.5500
        2.00     151.5500
        4.00      81.8583
        11.00    120.0000
        14.00    120.0000
                   ...   
3       61.00      6.2375
        63.00      9.5875
        65.00      7.7500
        70.50      7.7500
        74.00      7.7750
Name: fare, Length: 182, dtype: float64

In [5]:
## age 칼럼 정리
class_age = titanic.groupby(['pclass','age']).mean().fare.reset_index()
class_age.fare = class_age.fare.astype('int')
class_age.groupby(['pclass', 'fare']).mean().age.reset_index()
class_age

/tmp/ipykernel_1694/3714195747.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  class_age = titanic.groupby(['pclass','age']).mean().fare.reset_index()


,pclass,age,fare
0,1,0.92,151
1,1,2.00,151
2,1,4.00,81
3,1,11.00,120
4,1,14.00,120
...,...,...,...
177,3,61.00,6
178,3,63.00,9
179,3,65.00,7
180,3,70.50,7


In [6]:
# class_age.loc[(class_age.pclass == 3) & (class_age.fare == 8), 'age']

In [7]:
# def find_age(x):
#     try:
#         class_age.loc[(class_age.pclass == x[0]) & (class_age.fare == x[1]), 'age'].values[0]
#     except:
#         fare = np.min(np.abs(class_age.loc[(class_age.pclass == x[0]), 'fare'] - x[1]))
#         try:
#             fare = x[1] - fare
#             age = class_age.loc[(class_age.pclass == x[0]) & (class_age.fare == fare), 'age'].values[0]
#         except:
#             fare = x[1] + fare
#             age = class_age.loc[(class_age.pclass == x[0]) & (class_age.fare == fare), 'age'].values[0]
#     return age

In [8]:
def find_age(x):
    if ~np.isnan(x[2]):
        return x[2]
    else:
        dists=np.abs(class_age.loc[(class_age.pclass==x[0]), 'fare'] - x[1])
    # return dists
    return class_age.loc[dists[dists==np.min(dists)].index[0],'age']

In [9]:
titanic['cl_age'] = titanic.loc[:, ['pclass','fare', 'age']].apply(find_age, axis=1)

In [10]:
drop_cols = ['age', 'deck', 'class', 'alive', 'sex']
titanic.drop(columns = drop_cols, inplace=True)
titanic.isna().sum()

survived       0
pclass         0
sibsp          0
parch          0
fare           0
embarked       2
who            0
adult_male     0
embark_town    2
alone          0
cl_age         0
dtype: int64

In [11]:
titanic = titanic.loc[~titanic.embarked.isna(),:]

In [12]:
titanic.isna().sum()

survived       0
pclass         0
sibsp          0
parch          0
fare           0
embarked       0
who            0
adult_male     0
embark_town    0
alone          0
cl_age         0
dtype: int64

In [15]:
km = KMeans(6)
age_gr = km.fit(titanic.loc[:,['cl_age']]).labels_

/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f3de3a6b940>
Traceback (most recent call last):
  File "/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/bteam/anaconda3/envs/bteam

In [16]:
titanic['age_gr'] = age_gr

In [17]:
# titanic.dtypes

In [18]:
X = titanic.iloc[:,1:]
y = titanic.iloc[:,0]

In [19]:
object_cols = X.dtypes[X.dtypes == 'object'].index
object_cols

Index(['embarked', 'who', 'embark_town'], dtype='object')

In [20]:
for col in object_cols:
    encoder = LabelEncoder().fit_transform(X[col])
    X.drop(columns=col, inplace=True)
    X[col] = encoder

In [21]:
lor_model = LogisticRegression()
tree_model = DecisionTreeClassifier()
svm_model = SVC(probability=True)
ada_model = AdaBoostClassifier()

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
lor_model.fit(X_train, y_train)
tree_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)
ada_model.fit(X_train, y_train)

/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


AdaBoostClassifier()

In [23]:
# lr = lor_model.predict_proba(X_train)
# tree = tree_model.predict_proba(X_train)
# svm = svm_model.predict_proba(X_train)
# ada = ada_model.predict_proba(X_train)

lr = lor_model.predict(X_train)
tree = tree_model.predict(X_train)
svm = svm_model.predict(X_train)
ada = ada_model.predict(X_train)

In [24]:
meta_X = np.c_[lr, tree, svm, ada]

In [25]:
meta_model = RandomForestClassifier()
meta_model.fit(meta_X, y_train)
meta_X

array([[1, 0, 0, 1],
       [0, 0, 0, 0],
       [1, 1, 0, 1],
       ...,
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 0]])

In [26]:
# lr = lor_model.predict_proba(X_test)
# tree = tree_model.predict_proba(X_test)
# svm = svm_model.predict_proba(X_test)
# ada = ada_model.predict_proba(X_test)

lr = lor_model.predict(X_test)
tree = tree_model.predict(X_test)
svm = svm_model.predict(X_test)
ada = ada_model.predict(X_test)

In [27]:
meta_X = np.c_[lr, tree, svm, ada]

In [28]:
meta_model.score(meta_X, y_test)

0.757847533632287

In [29]:
class Stacking:
    def __init__(self,meta, *models):
        self.models = models
        self.meta_model = meta
        
    
    def fit(self, x, y):
        result_pred = np.zeros((len(x), len(self.models)))
        self.clfs = []
        for idx, model in enumerate(self.models):
            clf = model()
            self.clfs.append(clf.fit(x, y))
            result_pred[:,idx] = clf.predict(x)
            
        self.clf = self.meta_model()
        self.clf.fit(result_pred, y)
    
    def score(self, x, y):
        result_pred = np.zeros((len(x), len(self.models)))
        for idx, model in enumerate(self.clfs):
            result_pred[:,idx] = model.predict(x)
            
        return np.sum(self.clf.predict(result_pred)==y) / len(y)
    
    

In [30]:
stack = Stacking(AdaBoostClassifier, LogisticRegression, DecisionTreeClassifier, RandomForestClassifier)

In [31]:
stack.fit(X_train, y_train)

/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
stack.score(X_test, y_test)

0.8026905829596412

In [33]:
clfs = [stack.models[0]().fit(X_train, y_train)]

/home/bteam/anaconda3/envs/bteam/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
clfs.append(stack.models[1]().fit(X_train, y_train))

In [35]:
clfs

[LogisticRegression(), DecisionTreeClassifier()]

In [36]:
### 최적 meta값 찾기

In [157]:
class Stacking:
    def __init__(self,*models):
        self.models = np.array(models)

    def fit(self, x, y):
        X_train, X_test, y_train, y_test = train_test_split(x,y)
        self.model_score = {}
        for i in range(len(self.models)):
            meta_model = self.models[i]
            models = self.models[np.arange(len(self.models)) != i]
            result_pred = np.zeros((len(X_train), len(self.models)))
            result_pred_test = np.zeros((len(X_test), len(self.models)))
            self.clfs = []
            for idx, model in enumerate(models):
                clf = model()
                self.clfs.append(clf.fit(X_train,y_train))
                result_pred[:,idx] = clf.predict(X_train)
                result_pred_test[:,idx] = clf.predict(X_test)
            self.clf = meta_model()
            self.clf.fit(result_pred, y_train)
            score = self.clf.score(result_pred_test, y_test)
            self.model_score[self.models[i]] = score

        meta_idx = np.argmax(np.array(list(stack.model_score.values())))
        self.meta_model = self.models[meta_idx]
        self.models_=self.models[ np.arange(len(self.models)) !=meta_idx ]

        result_pred = np.zeros((len(x), len(self.models_)))

        self.best_models = []
        for idx, model in enumerate(self.models_):
            clf = model()
            clf.fit(x,y)
            self.best_models.append(clf)
            result_pred[:,idx] = clf.predict(x)
        self.meta_model = self.meta_model().fit(result_pred, y)


    def score(self, x, y):
        result_pred = np.zeros((len(x), len(self.best_models)))
        for idx, model in enumerate(self.best_models):
            result_pred[:,idx] = model.predict(x)
        score = np.sum(self.meta_model.predict(result_pred) ==y)/len(y)

        return score
    
    def predict(self, x):
        result_pred = np.zeros((len(x), len(self.best_models)))
        for idx, model in enumerate(self.best_models):
            result_pred[:,idx] = model.predict(x)
        return np.where(self.meta_model.predict(result_pred)==1, 'alive', 'dead')[0]


In [158]:
stack = Stacking(AdaBoostClassifier, LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, XGBClassifier, SVC)

In [159]:
# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [160]:
import warnings
warnings.filterwarnings(action="ignore")

In [161]:
stack.fit(X_train, y_train)

In [162]:
stack.score(X_test, y_test)

0.8340807174887892

In [163]:
stack.predict(X_test.iloc[[1],:])

'dead'

In [109]:
## predict_proba 적용

In [113]:
a = SVC()
a.fit(X_train, y_train)
a.predict(X_test).shape

(223,)

In [154]:
b=SVC(probability=True)
b.fit(X_train,y_train)
b.predict_proba(X_test).shape

(223, 2)

In [170]:
class Stacking:
    def __init__(self,*models):
        self.models = np.array(models)

    def fit(self, x, y):
        X_train, X_test, y_train, y_test = train_test_split(x,y)
        self.model_score = {}
        for i in range(len(self.models)):
            meta_model = self.models[i]
            models = self.models[np.arange(len(self.models)) != i]
            # result_pred = np.zeros((len(X_train), len(self.models)))
            # result_pred_test = np.zeros((len(X_test), len(self.models)))
            
            result_pred =np.zeros((len(X_train),1))
            result_pred_test = np.zeros((len(X_test),1))
            
            self.clfs = []
            for idx, model in enumerate(models):
                if 'SVC' in repr(model):
                    clf = model(property=True)
                else:
                    clf = model()
                self.clfs.append(clf.fit(X_train,y_train))
                result_pred = np.c_[result_pred, clf.predict_proba(X_train)]
                result_pred_test = np.c_[result_pred_test, clf.predict(X_test)]
            self.clf = meta_model()
            self.clf.fit(result_pred[:, 1:], y_train)
            score = self.clf.score(result_pred_test[:,1:], y_test)
            self.model_score[self.models[i]] = score

        meta_idx = np.argmax(np.array(list(stack.model_score.values())))
        self.meta_model = self.models[meta_idx]
        self.models_=self.models[ np.arange(len(self.models)) !=meta_idx ]

        result_pred = np.zeros((len(x), 1))

        self.best_models = []
        for idx, model in enumerate(self.models_):
            clf = model()
            clf.fit(x,y)
            self.best_models.append(clf)
            result_pred =np.c_[result_pred, clf.predict(x)]
        self.meta_model = self.meta_model().fit(result_pred[:,1:], y)


    def score(self, x, y):
        result_pred = np.zeros((len(x), len(self.best_models)))
        for idx, model in enumerate(self.best_models):
            result_pred[:,idx] = model.predict(x)
            
        score = np.sum(self.meta_model.predict(result_pred) ==y)/len(y)
        return score
    
    def predict(self, x):
        result_pred = np.zeros((len(x), len(self.best_models)))
        for idx, model in enumerate(self.best_models):
            result_pred[:,idx] = model.predict(x)
        np.where(self.meta_model.predict(result_pred)==1, 'alive', 'dead')[0]


In [171]:
stack = Stacking(AdaBoostClassifier, LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, XGBClassifier, SVC)

In [166]:
x = np.zeros((223,1))
x = np.c_[x,np.random.randn(223,2)]
x = np.c_[x,np.random.randn(223,2)]
x = x[:,1:]
x

array([[ 4.52912444e-01, -2.16638175e-02,  2.19473179e+00,
         1.62642115e-01],
       [ 1.02698489e+00,  3.23950776e-01,  6.23789398e-01,
         3.28199106e-01],
       [ 1.84230782e+00, -1.10657056e+00, -2.65080414e-01,
        -4.10218420e-01],
       [ 1.39328028e+00, -7.77684478e-01, -1.12915014e+00,
        -3.27659699e-01],
       [ 4.79983207e-01,  6.35537821e-01, -1.04680532e+00,
         1.88200197e+00],
       [-1.04488161e-01,  2.27962582e-01,  2.18250981e-01,
         7.48466202e-01],
       [ 1.41473200e-01, -1.12852338e+00, -1.07968765e+00,
         1.16009178e+00],
       [-1.47921243e-01,  6.68909156e-01,  1.22572546e+00,
        -9.67929543e-01],
       [ 2.96223900e-01, -2.74054633e-02,  4.93015310e-01,
         1.08390271e+00],
       [ 2.11999587e+00, -3.02731438e-01, -1.20926427e+00,
        -5.95099305e-01],
       [ 2.87115941e-01,  2.00067159e+00,  1.27035361e+00,
         1.32719045e+00],
       [ 1.01166050e+00,  7.40363590e-01,  7.23862547e-01,
      

In [172]:
from sklearn.datasets import load_iris

In [174]:
import pandas as pd
data = pd.DataFrame(load_iris()['data'])
data.columns = load_iris()['feature_names']
data['target'] = load_iris()['target']
data.to_csv('iris.csv', index=None)

In [176]:
clf = SVC()
try:
    clf.predict(X_test)
except Exception as e:
    print(e)

This SVC instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
